#### 原始的fragment2vec
- 选择parallel，best model
- 查看碎片之间的相似性
- 查看双键、芳香性等抽象度更高的结构信息

In [35]:
# Helper libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import matplotlib as mpl
# import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns

import time
from sklearn.manifold import TSNE
from sklearn.metrics import pairwise_distances

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_useSVG = True

In [2]:
def cal_distance(x, y, metric='euclidean'):
    if type(x) == pd.core.series.Series:
        x = x.values.reshape(1, -1)
    if type(y) == pd.core.series.Series:
        y = y.values.reshape(1, -1)
    return pairwise_distances(x, y, metric=metric)

In [3]:
def print_closest_words(x_embedding, x_query, n=5, add_vec=None):
    x = x_embedding.loc[x_query].values.reshape(1, -1).copy()
    # print('x is: {}'.format(x))
    if add_vec is not None:
        x += add_vec
        # print('x + add_vec is: {}'.format(x))
    dists = cal_distance(x=x_embedding.values, y=x)     # compute distances to all words
    lst = sorted(enumerate(dists), key=lambda x: x[1]) # sort by distance
    # print(lst[:100])
    all_smiles = []
    all_dis = [] 
    if add_vec is not None:
        for idx, difference in lst[0:n]:
            _smiles = x_embedding.iloc[idx,:].name
            all_smiles.append(_smiles)
            all_dis.append(difference[0])
            # print(_smiles, difference)
    else:
        for idx, difference in lst[1:n+1]:   # take the top n
            _smiles = x_embedding.iloc[idx,:].name
            all_smiles.append(_smiles)
            all_dis.append(difference[0])
            # print(_smiles, difference)
    return {'smiles': all_smiles, 'dis': all_dis}

In [4]:
def get_minus_result(x_embedding, x, y):
    x = x_embedding.loc[x].values.reshape(1, -1)
    y = x_embedding.loc[y].values.reshape(1, -1)
    return x-y

In [5]:
def draw_mol_by_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    size = (200, 200)
    return Draw.MolToImage(mol, size=size)

In [6]:
def draw_multiple_mol(smiles_list, mols_per_row=4, file_path=None, legends=None):
    mols = []
    for i in smiles_list:
        mols.append(Chem.MolFromSmiles(i))
    mols_per_row = min(len(smiles_list), mols_per_row)
    if legends is None:
        img=Draw.MolsToGridImage(mols, molsPerRow=mols_per_row, subImgSize=(220, 120), useSVG=True)
    else:
        img=Draw.MolsToGridImage(mols, molsPerRow=mols_per_row, subImgSize=(220, 120), useSVG=True, legends=legends)
    if file_path:
        with open(file_path, 'w') as f_handle:
            f_handle.write(img.data)
    return img

In [13]:
def show_each_md(x_reduced, frag_info, file_path=''):
    """
    reduced_x: 2 dimensions x with fragment as index, a dataframe
    frag_info: the number of each MD with fragemnt as index, a dataframe
    """
    # model = model_name
    fig, ax = plt.subplots(2, 4, figsize=(24, 12))
    ax = ax.flatten()
    # print(x_reduced.head(2))
    # print(frag_info.head(2))
    intersect_index = set(x_reduced.index.to_list()) & set(frag_info.index.to_list())
    x_reduced = x_reduced.loc[intersect_index, :].copy()  # alignment
    frag_info = frag_info.loc[intersect_index, :].copy()
    # reduced_x = reduced_x.loc[frag_info.index, :].copy()
    # parallel_frag_info = parallel_frag_info.loc[:, selected_md].copy()
    for i,md in enumerate(frag_info.columns.to_list()):
        # current_labels = parallel_frag_info.iloc[:, i]
        current_labels = frag_info.iloc[:, i]
        unique_labels = sorted(current_labels.unique())
        n_labels = len(unique_labels)
        # print(n_labels)
        cc = sns.color_palette('Blues', n_labels)
        for j,label in enumerate(unique_labels):
            current_nodes = (current_labels == label)
            ax[i].scatter(x_reduced.loc[current_nodes, 0], x_reduced.loc[current_nodes, 1],
                          c=colors.rgb2hex(cc[j]), vmin=0, vmax=10, s=10, label=str(label))
        ax[i].set_title(md, fontsize=12)
        ax[i].legend()
    plt.tight_layout()
    plt.savefig(file_path, bbox_inches='tight', transparent=True)
    plt.close()

In [30]:
def reduce_by_tsne(x):
    t0 = time.time()
    tsne = TSNE(n_components=2, n_jobs=4, learning_rate=200, early_exaggeration=20, n_iter=2000, random_state=42, init='pca', verbose=1)
    X_reduced_tsne = tsne.fit_transform(x)
    # X_reduced_tsne = tsne.fit(x)
    print(X_reduced_tsne.shape)
    # np.save('X_reduced_tsne_pca_first', X_reduced_tsne2)
    t1 = time.time()
    print("t-SNE took {:.1f}s.".format(t1 - t0))
    return X_reduced_tsne

In [7]:
frag2vec = pd.read_csv('./model_parallel/nn_trained_frag_embedding_reg.csv', index_col=0)
frag2vec.head(2)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
fragment,,,,,,,,,,,,,,,,,,,,,
CC,-0.558245,-1.438177,0.099167,-1.690964,0.609527,0.275151,-0.131262,-1.386071,-1.124998,-0.883890,...,0.888008,-1.322027,1.935024,2.144108,-0.883948,0.212668,0.572107,0.954083,-0.496624,-1.354179
CN,-0.458151,-1.483617,0.175892,-1.543361,0.740270,-0.030037,0.702270,-1.455249,-1.184582,-1.250804,...,0.650653,-1.315698,1.569044,3.021253,0.223890,0.342611,0.793450,0.123334,-1.163844,-1.464272


In [8]:
frag2vec.shape

(505, 30)

In [9]:
# demo_frags = np.random.choice(frag2vec.index.to_list(), 5, replace=False)
demo_frags = ['C1=CCNN=C1', 'C1=COCO1', 'C1=CCC1', 'C1=CN=N[SH]=C1', 'OBr']
demo_frags

['C1=CCNN=C1', 'C1=COCO1', 'C1=CCC1', 'C1=CN=N[SH]=C1', 'OBr']

In [10]:
frag2nn = {}
for frag in demo_frags:
    frag2nn[frag] = print_closest_words(x_embedding=frag2vec, x_query=frag, n=7)

In [11]:
nn1 = frag2nn['C1=COCO1']
nn1

{'smiles': ['C1=COCCO1',
  'C1=COCCCO1',
  'C1COCCO1',
  'C1=COCCOC1',
  'C1=CCOCOC1',
  'C1=COCOC1',
  'C1=NCCCO1'],
 'dis': [1.2642679190190445,
  2.2014491766004634,
  4.763376989660993,
  4.898847203456766,
  4.918049307447968,
  5.376133423346084,
  5.73322301711678]}

In [12]:
for k, frag in enumerate(demo_frags):
    nn = frag2nn[frag]
    draw_multiple_mol(smiles_list=[frag] + nn['smiles'],  
                   legends=[frag] + [nn['smiles'][i] + '(' + str('{:.2f}'.format(nn['dis'][i])) + ')' for i in range(7)],
                   file_path='./images/nn_trained_frag_nn_{}.svg'.format(k))

#### test double bond

In [14]:
double_bond = get_minus_result(x_embedding=frag2vec, x='C=O', y='CO')
# double_bond

In [15]:
single_bond_frag = ['CC', 'CN', 'CS', 'OS', 'C1CC1']
single_bond2nn = {}
for frag in single_bond_frag:
    single_bond2nn[frag] = print_closest_words(x_embedding=frag2vec, x_query=frag, n=4, add_vec=double_bond)

In [16]:
single_bond2nn['CC'], single_bond2nn['CN'], single_bond2nn['CS'], single_bond2nn['OS'], single_bond2nn['C1CC1']

({'smiles': ['C=O', 'C=C', 'CC', 'CF'],
  'dis': [3.57137566424523,
   6.220684831141861,
   6.26249704311534,
   6.711392594008851]},
 {'smiles': ['C=O', 'C=C', 'CC', 'CN'],
  'dis': [4.368489158483479,
   5.881544775989433,
   6.253310048210345,
   6.262497043115342]},
 {'smiles': ['C=O', 'C=S', 'CC', 'CS'],
  'dis': [4.620402986739358,
   5.671829338474722,
   5.991186776541636,
   6.262497043115342]},
 {'smiles': ['OS', 'C1=CSCCC1', 'C1=CSCCCC1', 'C1=CSCO1'],
  'dis': [6.262497043115342,
   8.417071350458937,
   8.464477290349604,
   8.481741662773006]},
 {'smiles': ['C1CC1', 'C1CCCCC1', 'C1=CCCCCC1', 'C1CCCNCC1'],
  'dis': [6.26249704311534,
   7.444596266454342,
   7.552632569772199,
   7.5989776165987255]})

#### test triple bond

In [17]:
tri_bond = get_minus_result(x_embedding=frag2vec, x='C#C', y='CC')

In [18]:
cn_add_tri_bond_nn = print_closest_words(x_embedding=frag2vec, x_query='CN', add_vec=tri_bond)
cn_add_tri_bond_nn

{'smiles': ['C#C', 'C#N', 'C', 'CC', 'CCl'],
 'dis': [2.8189291515517114,
  4.306643205063667,
  6.323543662681316,
  6.362918657382666,
  6.668080634743713]}

In [19]:
print_closest_words(x_embedding=frag2vec, x_query='CN')

{'smiles': ['CC', 'C', 'CO', 'C1=CC=CNC=C1', 'CBr'],
 'dis': [2.8189291515517154,
  3.1315868412713135,
  4.368489158483478,
  4.480302818442642,
  4.486963058649091]}

#### test aromaticity（芳香性）

In [20]:
arom = get_minus_result(x_embedding=frag2vec, x='C1=CC=CC=C1', y='C1CCCCC1')

In [21]:
print_closest_words(x_embedding=frag2vec, x_query='C1CCCC1')

{'smiles': ['C1CCCCC1', 'C1CCCCCC1', 'C1CCNCC1', 'C1=CCCNCC1', 'C1CCNC1'],
 'dis': [1.7803063772397651,
  2.185304887287405,
  2.6639290523041876,
  2.9731237331166986,
  3.0038468507528755]}

In [22]:
print_closest_words(x_embedding=frag2vec, x_query='C1CCCC1', add_vec=arom)

{'smiles': ['C1=CC=CC=C1',
  'C1=CC=CNC=C1',
  'C1=CC=CCC=C1',
  'C1=CCC=C1',
  'C1=CC=C1'],
 'dis': [1.7803063772397532,
  2.332125511660255,
  2.379351602688789,
  2.542296673378105,
  2.644370635495892]}

In [23]:
print_closest_words(x_embedding=frag2vec, x_query='C1CCNC1')

{'smiles': ['C1CCNCC1', 'C1CCCNCC1', 'C1CCCC1', 'C1CNC1', 'C1CC1'],
 'dis': [1.5067800684004364,
  2.917402829613875,
  3.0038468507528706,
  3.111474152976513,
  3.30667100767897]}

In [24]:
print_closest_words(x_embedding=frag2vec, x_query='C1CCNC1', add_vec=arom)

{'smiles': ['C1=CC=CNC=C1',
  'C1=CC=NC=C1',
  'C1=CC=CC=C1',
  'C1=CCC=NC=C1',
  'C1=CC=CCC=C1'],
 'dis': [3.02818957860175,
  3.3719614130850677,
  3.4594979504910883,
  3.921795515515275,
  3.9824740691580347]}

#### test N atom

In [25]:
n = get_minus_result(x_embedding=frag2vec, x='C1CNC1', y='C1CC1')

In [26]:
print_closest_words(x_embedding=frag2vec, x_query='C1CCCC1')

{'smiles': ['C1CCCCC1', 'C1CCCCCC1', 'C1CCNCC1', 'C1=CCCNCC1', 'C1CCNC1'],
 'dis': [1.7803063772397651,
  2.185304887287405,
  2.6639290523041876,
  2.9731237331166986,
  3.0038468507528755]}

In [27]:
print_closest_words(x_embedding=frag2vec, x_query='C1CCCC1', add_vec=n)

{'smiles': ['C1CNC1', 'C1CCCC1', 'C1=CC2CCC1CN2', 'C1=CC2CC1CN2', 'C1CCNCC1'],
 'dis': [3.578308449144305,
  4.900267785794218,
  4.995280872821142,
  5.145661406014962,
  5.418627289758258]}

#### show md

In [29]:
# md4class = ['nN', 'nS', 'nO', 'nBondsD', 'naRing', 'nARing']
parallel_frag_info = pd.read_csv('../fragment2vec/step3_model_parallel2vec.bin_frag_info.csv', index_col=0)
# parallel_frag_info = parallel_frag_info.loc[:, md4class].copy()
parallel_frag_info.head(2)

,nN,nS,nO,nX,nBondsD,nBondsT,naRing,nARing
fragment,,,,,,,,
CC,0,0,0,0,0,0,0,0
CN,1,0,0,0,0,0,0,0


In [33]:
x_2d = reduce_by_tsne(frag2vec)
x_2d_df = pd.DataFrame(data=x_2d, index=frag2vec.index)
x_2d_df.head(2)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 505 samples in 0.010s...
[t-SNE] Computed neighbors for 505 samples in 0.114s...
[t-SNE] Computed conditional probabilities for sample 505 / 505
[t-SNE] Mean sigma: 2.423002
[t-SNE] KL divergence after 250 iterations with early exaggeration: 124.167282
[t-SNE] KL divergence after 1750 iterations: 0.528396
(505, 2)
t-SNE took 3.9s.


,0,1
fragment,,
CC,-22.826437,4.778559
CN,-22.517534,4.215286


In [36]:
show_each_md(x_reduced=x_2d_df, frag_info=parallel_frag_info, file_path='./images/t-sne_md_after_training.png')